In [1]:
from pathlib import Path
import subprocess

In [2]:
raw_datadir = Path('../data/raw/')

In [3]:
filepath = raw_datadir / 'math.stackexchange.com.7z'
print(filepath)

../data/raw/math.stackexchange.com.7z


In [85]:
def get_output_dirname(path):
    filename = path.name
    return path.with_suffix('')
    
def open_7z(path):
    output_dirname = get_output_dirname(path)
    output_dirname.mkdir(exist_ok=True, parents=True)
    exe_str = f"7za e {path} -o{output_dirname}"
    
    print(f'Reading data from {path}')
    print(f'Saving data to {output_dirname}')
    print("Running:", exe_str)
    p = subprocess.call(exe_str, shell=True)
    print(f'Done. {p}')
    return

In [88]:
open_7z(filepath)

Reading data from ../data/raw/math.stackexchange.com.7z
Saving data to ../data/raw/math.stackexchange.com
Running: 7za e ../data/raw/math.stackexchange.com.7z -o../data/raw/math.stackexchange.com
Done. 0


In [89]:
!ls ../data/raw/math.stackexchange.com

Badges.xml      PostHistory.xml Posts.xml       Users.xml
Comments.xml    PostLinks.xml   Tags.xml        Votes.xml


In [22]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_xml_file(path):
    """Read the xml file from path and return a pandas dataframe with all the data."""
    root = ET.parse(path).getroot()
    rows = root.findall('row')
    return pd.DataFrame([dict(row.items()) for row in rows])

In [25]:
df = parse_xml_file('../data/raw/math.stackexchange.com/Comments.xml')

In [27]:
df.head()

,CreationDate,Id,PostId,Score,Text,UserDisplayName,UserId
0,2010-07-20T19:24:44.963,2,9,6,Fantastic answer!,NaN,10
1,2010-07-20T19:26:09.357,3,9,6,"I like this so far, but maybe add a bit on unc...",NaN,16
2,2010-07-20T19:27:07.983,5,17,1,"Oh, I figured as so. I remember learning about...",NaN,40
3,2010-07-20T19:28:16.077,8,17,1,It is much better than an approximation: it gi...,NaN,55
4,2010-07-20T19:29:17.900,10,20,3,You've missed out many types of number: comple...,NaN,35
